In [1]:
import os
import time
import math
import random
import datetime
from pathlib import Path
!pip install transformers
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"  # reduce the amount of console output from TF
import tensorflow as tf
from transformers import *
!pip install -q datasets # install HF datasets library
from datasets import load_dataset
import torch

logging.set_verbosity_warning()
logging.set_verbosity_error()

import logging

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(


In [2]:
print('TF version',tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU'))) # check GPU available

TF version 2.12.0
Num GPUs Available:  1


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Load Dataset

In [4]:
dataset_full = load_dataset("mbpp")

  0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
test_set = dataset_full['test']

In [6]:
test_set[3]

{'task_id': 14,
 'text': 'Write a python function to find the volume of a triangular prism.',
 'code': 'def find_Volume(l,b,h) : \r\n    return ((l * b * h) / 2) ',
 'test_list': ['assert find_Volume(10,8,6) == 240',
  'assert find_Volume(3,2,2) == 6',
  'assert find_Volume(1,2,1) == 1'],
 'test_setup_code': '',
 'challenge_test_list': []}

# Model codet5-large-ntp-py

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-large-ntp-py")

In [8]:
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-large-ntp-py").to(device);

In [9]:
# def make_prediction(model, input_ids: list, max_length=128):
#   outputs = model.generate(input_ids, max_length=max_length)
#   return tokenizer.decode(outputs[0], skip_special_tokens=True)

- Example 1: Few-shot

In [21]:
text = """Write a python function to remove first and last occurrence of a given character from the string. Your code should satisfy these tests:\n assert remove_Occ("hello","l") == "heo"\n assert remove_Occ("abcda","a") == "bcd"\n assert remove_Occ("PHP","P") == "H" """

In [22]:
text

'Write a python function to remove first and last occurrence of a given character from the string. Your code should satisfy these tests:\n assert remove_Occ("hello","l") == "heo"\n assert remove_Occ("abcda","a") == "bcd"\n assert remove_Occ("PHP","P") == "H" '

In [23]:
model_inputs = tokenizer(text, return_tensors="pt")

- Example 2: Few-shot

In [ ]:
text = test_set[1]['text'] + ' Your code should satisfy these tests:\n' +'\n'.join(test_set[1]['test_list'])

In [ ]:
text

'Write a function to sort a given matrix in ascending order according to the sum of its rows. Your code should satisfy these tests:\nassert sort_matrix([[1, 2, 3], [2, 4, 5], [1, 1, 1]])==[[1, 1, 1], [1, 2, 3], [2, 4, 5]]\nassert sort_matrix([[1, 2, 3], [-2, 4, -5], [1, -1, 1]])==[[-2, 4, -5], [1, -1, 1], [1, 2, 3]]\nassert sort_matrix([[5,8,9],[6,4,3],[2,1,4]])==[[2, 1, 4], [6, 4, 3], [5, 8, 9]]'

- Example 3: Pass@k

In [14]:
text = """Write a python function to remove first and last occurrence of a given character from the string. Your code should satisfy these tests:\n assert remove_Occ("hello","l") == "heo"\n assert remove_Occ("abcda","a") == "bcd"\n assert remove_Occ("PHP","P") == "H" """

In [26]:
%%time
model_inputs = tokenizer(text, return_tensors="pt").to(device)

CPU times: user 1.08 ms, sys: 18 µs, total: 1.1 ms
Wall time: 1.12 ms


In [27]:
# output_1 = model.generate(model_inputs.input_ids, max_length=128)

In [28]:
%%time
output_2 = model.generate(model_inputs.input_ids, max_length=128, num_beams=80, num_return_sequences=80, early_stopping=True)
  # return tokenizer.decode(outputs[0], skip_special_tokens=True)

CPU times: user 2.7 s, sys: 58.7 ms, total: 2.76 s
Wall time: 2.87 s


In [25]:
%%time
output_2 = model.generate(model_inputs.input_ids, max_length=128, num_beams=80, num_return_sequences=80, early_stopping=True)
  # return tokenizer.decode(outputs[0], skip_special_tokens=True)

CPU times: user 1min 41s, sys: 1.22 s, total: 1min 42s
Wall time: 2min 10s


In [74]:
# # now we have 3 output sequences
# print("Output:\n")
# list_of_outputs = []
# for i, beam_output in enumerate(output_2):
#   output = tokenizer.decode(beam_output, skip_special_tokens=True)
#   list_of_outputs.append(output)
#   print("{}: {}".format(i, output))

In [ ]:
# exec("""def remove_Occ(string, char):
#     return string[1:-1].replace(char, "")""")

In [ ]:
# exec('assert remove_Occ("hello","l") == "heo"')

# Prediction

In [10]:
import csv
import pandas as pd
import os

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
# Max length on test set
max([len(i['code'])for i in test_set])

1331

In [13]:
def make_prediction(model, input_ids: list, max_length=128, batch=False,**kwargs):
  outputs = model.generate(input_ids, max_length=max_length, **kwargs)
  if batch:
    outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  else:
    outputs = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return outputs

In [14]:
def save_output(output_file: str, data: list, header):
  # Create file if it doesn't exist
  if not os.path.exists(output_file):
    with open(output_file, "a+", encoding='utf-8') as f:
        write = csv.writer(f)
        write.writerows([header])
  results_df = pd.DataFrame.from_records(data, columns=header)
  results_df.to_csv(output_file, mode='a', index=False, header=None)

- Single Prection

In [ ]:
# MAX_LENGTH = 1331
results = []
# list_of_inputs = []
# list_of_true_labels = []
# list_of_predictions = []
full_path = '/content/drive/MyDrive/CSCI544/Project'
model_name = 'codet5-large-ntp-py'
output_file = f"{full_path}/{model_name}-pred.csv"
for sample in test_set:
  task_id = sample['task_id']
  example = sample['text']
  true_label = sample['code']
  max_length = len(true_label)
  inputs = example + ' Your code should satisfy these tests:\n ' +'\n '.join(sample['test_list'])
  model_inputs = tokenizer(inputs, return_tensors="pt") # prediction with same lengh as output
  output = make_prediction(model, model_inputs.input_ids, max_length=max_length)
  # list_of_inputs.append([inputs]) 
  # list_of_predictions.append([output])
  # list_of_true_labels.append([true_label])
  result = [{'task_id': task_id, 'text': example, 'test_list': sample['test_list'],
             'input': inputs, 'true_label': true_label, 'prediction': output}]
  results.append(result)
  save_output(output_file, result)
  # break

- Multiple Predictions (num_return_sequences)

In [15]:
print('Model on GPU?: ', next(model.parameters()).is_cuda)

Model on GPU?:  True


In [16]:
# MAX_LENGTH = 1331
results = []
# list_of_inputs = []
# list_of_true_labels = []
# list_of_predictions = []
full_path = '/content/gdrive/MyDrive/CSCI544/Project'
model_name = 'codet5-large-ntp-py-multiple-sentences-2'
output_file = f"{full_path}/{model_name}-pred.csv"
for sample in test_set:
  task_id = sample['task_id']
  example = sample['text']
  true_label = sample['code']
  max_length = len(true_label)
  inputs = example + ' Your code should satisfy these tests:\n ' +'\n '.join(sample['test_list'])
  model_inputs = tokenizer(inputs, return_tensors="pt").to(device) # prediction with same lengh as output
  output = make_prediction(model, model_inputs.input_ids, max_length=max_length, batch=True,
                           num_beams=80, num_return_sequences=80, early_stopping=True)
  del model_inputs
  # list_of_inputs.append([inputs]) 
  # list_of_predictions.append([output])
  # list_of_true_labels.append([true_label])
  result = [{'task_id': task_id, 'text': example, 'test_list': sample['test_list'],
             'input': inputs, 'true_label': true_label, 'prediction': output}]
  results.append(result)
  save_output(output_file, result, header=['task_id', 'text', 'test_list',
             'input', 'true_label', 'prediction'])

OutOfMemoryError: ignored

# Evaluation

In [115]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np
import pandas as pd
import logging

logger = logging.getLogger('Evaluation')

logger.setLevel(logging.DEBUG)# allow DEBUG level messages to pass through the logger

In [116]:
import sys

recursion_limit = sys.getrecursionlimit()
print(f"Current recursion limit: {recursion_limit}")

Current recursion limit: 1000


In [65]:
nltk.download('punkt');

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [66]:
full_path = '/content/drive/MyDrive/CSCI544/Project'
model_name = 'codet5-large-ntp-py'
output_file = f"{full_path}/{model_name}-pred.csv"

In [67]:
df = pd.read_csv('/content/drive/MyDrive/CSCI544/Project/codet5-large-ntp-py-pred.csv')

In [ ]:
sentences = df[['true_label', 'prediction']].values

- Bleu Score

In [ ]:
def compute_bleu_score(sentences: np.array) -> float:
  n_rows = sentences.shape[0]
  bleu_scores = []
  max_ngram_size = 4

  # define the smoothing function
  smooth_fn = SmoothingFunction() # Smoothing method 1: Add epsilon counts to precision with 0 counts.

  for i in range(n_rows):
    reference = sentences[i][0] # true label
    candidate = sentences[i][1] # prediction
    # Tokenize
    true_token = word_tokenize(reference)
    pred_token = word_tokenize(candidate)
    score = nltk.translate.bleu_score.sentence_bleu(
        [true_token], pred_token, weights=[1./max_ngram_size]*max_ngram_size,
        smoothing_function=smooth_fn.method1)
    bleu_scores.append(score)

  # compute the average BLEU score over all candidate sentences
  avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
  return avg_bleu_score


In [ ]:
print('Bleu Score: ', compute_bleu_score(sentences))

Bleu Score:  0.10143968640805599


- Synthesis Performance

In [ ]:
rows = df[['task_id', 'test_list','true_label', 'prediction']].values

In [ ]:
exec('assert 2 == 2')

In [ ]:
acc_list = []
error_messages = []
result = []
logger.info('Starting loop')

for i, row in enumerate(rows):
  task_id, test_list, true_label, prediction = row
  logger.info(f'Processing item {i+1} of {len(rows)}. Task_id: {task_id}')
  test_list = eval(test_list) # evaluate the string representation of each list as a Python expression
  # Get code after 'def'
  def_index = prediction.find('def')  # find the index of the 'def' keyword
  function_def = prediction[def_index:]  # extract the substring starting from 'def'

  error_list = []
  count_success = 0
  try:
    if task_id != 45:   
      exec(function_def)
      for i, test in enumerate(test_list):
        try:
          exec(test)
          error_list.append(f'Test {i} Success')
          count_success += 1
        except Exception as e:
          # print(type(e))
          error_list.append(f'Test {i} {type(e)}: {e}')
    else:
      error_list.append(f'Test {i}: Recursion problem')

  except Exception as e:
    error_list.append(f'{type(e)}: {e}')
  acc_list.append(count_success/len(test_list))
  error_messages.append(error_list)
  result.append(
      {'task_id': task_id, 'test_list': test_list, 'true_label': true_label,
       'prediction': prediction, 'successful_test_cases': count_success,
       'error_list': error_list})


INFO:Evaluation:Starting loop
INFO:Evaluation:Processing item 1 of 500. Task_id: 11
INFO:Evaluation:Processing item 2 of 500. Task_id: 12
INFO:Evaluation:Processing item 3 of 500. Task_id: 13
INFO:Evaluation:Processing item 4 of 500. Task_id: 14
INFO:Evaluation:Processing item 5 of 500. Task_id: 15
INFO:Evaluation:Processing item 6 of 500. Task_id: 16
INFO:Evaluation:Processing item 7 of 500. Task_id: 17
INFO:Evaluation:Processing item 8 of 500. Task_id: 18
INFO:Evaluation:Processing item 9 of 500. Task_id: 19
INFO:Evaluation:Processing item 10 of 500. Task_id: 20
INFO:Evaluation:Processing item 11 of 500. Task_id: 21
INFO:Evaluation:Processing item 12 of 500. Task_id: 22
INFO:Evaluation:Processing item 13 of 500. Task_id: 23
INFO:Evaluation:Processing item 14 of 500. Task_id: 24
INFO:Evaluation:Processing item 15 of 500. Task_id: 25
INFO:Evaluation:Processing item 16 of 500. Task_id: 26
INFO:Evaluation:Processing item 17 of 500. Task_id: 27
INFO:Evaluation:Processing item 18 of 500. T

0
1
0
10090807060504030201
1
20019018017016015014013012011010090807060504030201


INFO:Evaluation:Processing item 111 of 500. Task_id: 121
INFO:Evaluation:Processing item 112 of 500. Task_id: 122
INFO:Evaluation:Processing item 113 of 500. Task_id: 123
INFO:Evaluation:Processing item 114 of 500. Task_id: 124
INFO:Evaluation:Processing item 115 of 500. Task_id: 125
INFO:Evaluation:Processing item 116 of 500. Task_id: 126
INFO:Evaluation:Processing item 117 of 500. Task_id: 127
INFO:Evaluation:Processing item 118 of 500. Task_id: 128
INFO:Evaluation:Processing item 119 of 500. Task_id: 129
INFO:Evaluation:Processing item 120 of 500. Task_id: 130
INFO:Evaluation:Processing item 121 of 500. Task_id: 131
INFO:Evaluation:Processing item 122 of 500. Task_id: 132
INFO:Evaluation:Processing item 123 of 500. Task_id: 133
INFO:Evaluation:Processing item 124 of 500. Task_id: 134
INFO:Evaluation:Processing item 125 of 500. Task_id: 135
INFO:Evaluation:Processing item 126 of 500. Task_id: 136
INFO:Evaluation:Processing item 127 of 500. Task_id: 137
INFO:Evaluation:Processing item

False
False
False
2
0
0
False
False
False


INFO:Evaluation:Processing item 232 of 500. Task_id: 242
INFO:Evaluation:Processing item 233 of 500. Task_id: 243
INFO:Evaluation:Processing item 234 of 500. Task_id: 244
INFO:Evaluation:Processing item 235 of 500. Task_id: 245
INFO:Evaluation:Processing item 236 of 500. Task_id: 246
INFO:Evaluation:Processing item 237 of 500. Task_id: 247
INFO:Evaluation:Processing item 238 of 500. Task_id: 248
INFO:Evaluation:Processing item 239 of 500. Task_id: 249
INFO:Evaluation:Processing item 240 of 500. Task_id: 250
INFO:Evaluation:Processing item 241 of 500. Task_id: 251
INFO:Evaluation:Processing item 242 of 500. Task_id: 252
INFO:Evaluation:Processing item 243 of 500. Task_id: 253
INFO:Evaluation:Processing item 244 of 500. Task_id: 254
INFO:Evaluation:Processing item 245 of 500. Task_id: 255
INFO:Evaluation:Processing item 246 of 500. Task_id: 256
INFO:Evaluation:Processing item 247 of 500. Task_id: 257
INFO:Evaluation:Processing item 248 of 500. Task_id: 258
INFO:Evaluation:Processing item

44
74
18
2
1
1


INFO:Evaluation:Processing item 396 of 500. Task_id: 406
INFO:Evaluation:Processing item 397 of 500. Task_id: 407
INFO:Evaluation:Processing item 398 of 500. Task_id: 408
INFO:Evaluation:Processing item 399 of 500. Task_id: 409
INFO:Evaluation:Processing item 400 of 500. Task_id: 410
INFO:Evaluation:Processing item 401 of 500. Task_id: 411
INFO:Evaluation:Processing item 402 of 500. Task_id: 412
INFO:Evaluation:Processing item 403 of 500. Task_id: 413
INFO:Evaluation:Processing item 404 of 500. Task_id: 414
INFO:Evaluation:Processing item 405 of 500. Task_id: 415
INFO:Evaluation:Processing item 406 of 500. Task_id: 416
INFO:Evaluation:Processing item 407 of 500. Task_id: 417
INFO:Evaluation:Processing item 408 of 500. Task_id: 418
INFO:Evaluation:Processing item 409 of 500. Task_id: 419
INFO:Evaluation:Processing item 410 of 500. Task_id: 420
INFO:Evaluation:Processing item 411 of 500. Task_id: 421
INFO:Evaluation:Processing item 412 of 500. Task_id: 422
INFO:Evaluation:Processing item

[23, 12, 5, 12, 4, 81, 3, 56, 2, 95]
[39, 34, 87, 24, 73, 22, 68]
[96, 32, 82, 30, 83, 16, 74]
-1 is negative
4 is not negative
5 is not negative
-6 is negative
-1 is negative
4 is not negative
5 is not negative
-6 is negative
-1 is negative
-2 is negative
3 is not negative
4 is not negative
-7 is negative
-6 is negative
8 is not negative
9 is not negative
False
False
False
pollgon
aharaater
python


INFO:Evaluation:Processing item 475 of 500. Task_id: 485
INFO:Evaluation:Processing item 476 of 500. Task_id: 486
INFO:Evaluation:Processing item 477 of 500. Task_id: 487
INFO:Evaluation:Processing item 478 of 500. Task_id: 488
INFO:Evaluation:Processing item 479 of 500. Task_id: 489
INFO:Evaluation:Processing item 480 of 500. Task_id: 490
INFO:Evaluation:Processing item 481 of 500. Task_id: 491
INFO:Evaluation:Processing item 482 of 500. Task_id: 492
INFO:Evaluation:Processing item 483 of 500. Task_id: 493
INFO:Evaluation:Processing item 484 of 500. Task_id: 494
INFO:Evaluation:Processing item 485 of 500. Task_id: 495
INFO:Evaluation:Processing item 486 of 500. Task_id: 496
INFO:Evaluation:Processing item 487 of 500. Task_id: 497
INFO:Evaluation:Processing item 488 of 500. Task_id: 498
INFO:Evaluation:Processing item 489 of 500. Task_id: 499
INFO:Evaluation:Processing item 490 of 500. Task_id: 500
INFO:Evaluation:Processing item 491 of 500. Task_id: 501
INFO:Evaluation:Processing item

### Compute metric

- Passing all 3 test cases

In [ ]:
acc_all_test_cases = [1 if acc == 1 else 0 for acc in acc_list]

In [ ]:
import builtins

In [ ]:
print(f'Tasks Solved: {builtins.sum(acc_all_test_cases)/len(acc_all_test_cases)*100:.2f}%')

Tasks Solved: 14.40%


- Passing at least 2 test cases

In [ ]:
acc_all_test_cases = [1 if acc > 0.5 else 0 for acc in acc_list]

In [ ]:
print(f'Tasks Solved: {builtins.sum(acc_all_test_cases)/len(acc_all_test_cases)*100:.2f}%')

Tasks Solved: 19.00%


- Passing at least 1 test case

In [ ]:
acc_all_test_cases = [1 if acc > 0 else 0 for acc in acc_list]

In [ ]:
print(f'Tasks Solved: {builtins.sum(acc_all_test_cases)/len(acc_all_test_cases)*100:.2f}%')

Tasks Solved: 26.40%


### Pass @k

The pass@k is the fraction of prompts for which the model succeeded in this sense.

For example, if Pass@5 is 80%, it means that for 80% of the queries, the system returned at least one relevant document among the top 5 results.

In [38]:
import os
os.environ["HF_ALLOW_CODE_EVAL"] = "1"

In [40]:
import locale
locale.getpreferredencoding = lambda: "UTF-8" # => solve colab error: NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [41]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.8 MB/s eta 0:00:00


In [147]:
def compute_pass_k_metric(test_cases: list[list], candidates: list[list], k: list) -> tuple[dict, dict]:
  '''The Code Eval metric calculates how good are predictions given a set of references
  https://huggingface.co/spaces/evaluate-metric/code_eval

  Args: 
    test_cases
    candidates
    k
  Returns:
    (dict, dict)
  '''
  # Join test cases into a single string.
  test_cases_joined = ['\n'.join(test) for test in test_cases]
  pass_at_k, results = code_eval.compute(references=test_cases_joined, predictions=candidates, k=k)
  return pass_at_k, results

In [126]:
df = pd.read_csv('/content/gdrive/MyDrive/CSCI544/Project/codet5-large-ntp-py-multiple-sentences-pred.csv')

In [142]:
pass_k_test = []
pass_k_predictions = []
for test, pred in zip(df['test_list'], df['prediction']):
    pass_k_test.append(eval(test))
    pass_k_predictions.append(eval(pred))

In [145]:
pass_k_test[0]

['assert remove_Occ("hello","l") == "heo"',
 'assert remove_Occ("abcda","a") == "bcd"',
 'assert remove_Occ("PHP","P") == "H"']

In [149]:
pass_at_k, results = compute_pass_k_metric(pass_k_test, pass_k_predictions, [1,2,5, 80])

In [151]:
print(pass_at_k)

{'pass@1': 0.0, 'pass@2': 0.0, 'pass@5': 0.0, 'pass@80': 0.0}


In [150]:
# a = [i[i.find('def'):] for i in list_of_outputs]

In [45]:
# from evaluate import load
# code_eval = load("code_eval")
# test_cases = ["assert add(2,3)==5"]
# candidates = [["def add(a,b): return a*b", "def add(a, b): return a+b", "def add(a, b): return a-b"]]
# test_cases_2 = ['assert remove_Occ("hello","l") == "heo"\nassert remove_Occ("abcda","a") == "bcd"\nassert remove_Occ("PHP","P") == "H"']
# candidates_2 = [a]

In [ ]:
# from evaluate import load
# code_eval = load("code_eval")
# test_cases = ["assert add(2,3)==5"]
# candidates = [["def add(a,b): return a*b", "def add(a, b): return a+b", "def add(a, b): return a-b"]]
# test_cases_2 = ['assert remove_Occ("hello","l") == "heo"\nassert remove_Occ("abcda","a") == "bcd"\nassert remove_Occ("PHP","P") == "H"']
# candidates_2 = [a[:-1]]

ModuleNotFoundError: ignored

# TO DO:

 - [x] Compare performance with 0, 1, 2 or 3 test cases
 - [ ] Split error types in: CompileError, RuntimeError, FailedTest, and PassedTest